In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import datashader as ds
import panel as pn

from colorcet import bmy
from holoviews.operation.datashader import rasterize, datashade

hv.extension('bokeh')

def load_data():
    df = pd.read_csv('data/oggm_glacier_explorer.csv')
    df['latdeg'] = df.cenlat
    df['x'], df['y'] = ds.utils.lnglat_to_meters(df.cenlon, df.cenlat)
    return df

df = pn.state.as_cached('glaciers', load_data)

df.tail()

data = hv.Dataset(df, [('x', 'Longitude'), ('y', 'Latitude')],
                     [('avg_prcp', 'Annual Precipitation (mm/yr)'),
                      ('area_km2', 'Area'), ('latdeg', 'Latitude (deg)'),
                      ('avg_temp_at_mean_elev', 'Annual Temperature at avg. altitude'), 
                      ('mean_elev', 'Elevation')])
total_area = df.area_km2.sum()

print(data, len(data), total_area)

geo_kw    = dict(aggregator=ds.sum('area_km2'), x_sampling=1000, y_sampling=1000)
elev_kw   = dict(cmap='#7d3c98')
temp_kw   = dict(num_bins=50, adjoin=False, normed=False, bin_range=data.range('avg_temp_at_mean_elev'))
prcp_kw   = dict(num_bins=50, adjoin=False, normed=False, bin_range=data.range('avg_prcp'))

size_opts = dict(min_height=400, min_width=600, responsive=True)
geo_opts  = dict(size_opts, cmap=bmy, logz=True, colorbar=True, xlabel='', ylabel='')
elev_opts = dict(size_opts, show_grid=True)
temp_opts = dict(size_opts, fill_color='#f1948a', default_tools=[], toolbar=None, alpha=1.0)
prcp_opts = dict(size_opts, fill_color='#85c1e9', default_tools=[], toolbar=None, alpha=1.0)


geo_bg = hv.element.tiles.EsriImagery().opts(alpha=0.6, bgcolor="black")
geopoints = hv.Points(data, vdims=['area_km2']).opts(**geo_opts)

(geo_bg*rasterize(geopoints, **geo_kw).options(**geo_opts) + 
 datashade(data.to(hv.Scatter, 'mean_elev','latdeg', []), **elev_kw).options(**elev_opts) + 
 data.hist('avg_temp_at_mean_elev', **temp_kw).options(**temp_opts) +
 data.hist('avg_prcp',              **prcp_kw).options(**prcp_opts)).cols(2)